In [1]:
!pip install emoji


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [15]:
import pandas as pd
import numpy as np

In [2]:
import emoji

emoji_dictionary = {"0": "\u2764\uFE0F",
                    "1": ":baseball:",
                    "2": ":smile:",
                    "3": ":disappointed:",
                    "4": ":fork_and_knife:"}

def label_to_emoji(label):

    return emoji.emojize(emoji_dictionary[str(label)],language='alias')

In [4]:
dir = '/home/milad/Desktop/python/DL/Mini_project/db/emojify'
df=pd.read_csv(f'{dir}/train_emoji.csv',header=None)
df.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [5]:
X_train,Y_train=df[0].values,df[1].values

In [6]:
test_df=pd.read_csv(f'{dir}/test_emoji.csv',header=None)
X_test,Y_test=test_df[0].values,test_df[1].values

In [7]:
X_test.shape,Y_test.shape

((56,), (56,))

In [8]:
index=2

print(X_train[index],label_to_emoji(Y_train[index]))

It is the worst day in my life 😞


In [9]:
from tensorflow.keras.utils import to_categorical

Y_train_oh=to_categorical(Y_train,5)
Y_test_oh=to_categorical(Y_test,5)

2024-07-28 19:17:59.725605: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 19:17:59.730327: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 19:17:59.794931: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-28 19:18:01.398727: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
Y_test_oh[1]

array([0., 0., 0., 1., 0.])

In [11]:
X_train[0]

'never talk to me again'

In [12]:
max(X_train,key=len),len(max(X_train,key=len))

('I am so impressed by your dedication to this project', 52)

In [13]:
max_len=len(max(X_train,key=len).split())
max_len

10

In [16]:
dir='/home/milad/Desktop/python/DL/Mini_project/db/glove.6B'
embeddings_index={}
words_to_index={}
index_to_words={}
with open(f'{dir}/glove.6B.100d.txt',encoding='utf-8') as f:
  lines=f.readlines()
  words =set()
  for line in lines:
    values=line.split()
    word=values[0]
    embed=values[1:]
    weights=np.asarray(embed,dtype='float32')
    embeddings_index[word]=weights
    words.add(word)

  i = 1
  for w in sorted(words):
    words_to_index[w]=i
    index_to_words[i]=w
    i+=1

In [17]:
def senetences_to_indices(X,word_to_index,max_len):
  m=X.shape[0]
  X_indices=np.zeros((m,max_len))

  for i in range(m):
    sentence_words=X[i].lower().split()
    for j,w in enumerate(sentence_words):
      X_indices[i,j]=words_to_index[w]

  return X_indices

In [18]:
X_train_sequences=senetences_to_indices(X_train,words_to_index,max_len=max_len)
X_train[1],X_train_sequences[1]

('I am proud of your achievements',
 array([185457.,  52943., 293982., 268046., 394565.,  45460.,      0.,
             0.,      0.,      0.]))

In [22]:
from tensorflow.keras.layers import Input,AveragePooling1D,Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding

In [23]:
def pretrained_embedding_layer(embeddings_index,words_to_index):
  vocab_len = len(words_to_index) + 1 # oov
  emb_dim=embeddings_index['ali'].shape[0]
  emb_matrix=np.zeros((vocab_len,emb_dim))
  for word,index in words_to_index.items():
    emb_matrix[index,:]=embeddings_index[word]


  embedding_layer = Embedding(vocab_len,emb_dim,trainable=False)
  embedding_layer.build((None,))
  embedding_layer.set_weights([emb_matrix])

  return embedding_layer

In [24]:
def EmojifyModel(input_shape,embeddings_index,words_to_index,max_len=10):
  sentence_indices=Input(input_shape,dtype=np.int32)
  embedding_layer=pretrained_embedding_layer(embeddings_index,words_to_index)
  embeddings=embedding_layer(sentence_indices)
  x=AveragePooling1D(pool_size=max_len)(embeddings)
  x=Flatten()(x)

  x=Dense(5,activation='softmax')(x)
  model=Model(sentence_indices,x)
  return model

In [25]:
model = EmojifyModel((max_len,),embeddings_index,words_to_index)
model.summary()

2024-07-28 19:22:34.022312: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 160000400 exceeds 10% of free system memory.
2024-07-28 19:22:34.217736: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 160000400 exceeds 10% of free system memory.
2024-07-28 19:22:34.285177: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 160000400 exceeds 10% of free system memory.
2024-07-28 19:22:34.976465: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 160000400 exceeds 10% of free system memory.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 10, 100)        │    40,000,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d               │ (None, 1, 100)         │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           505 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,000,605 (152.59 MB)

 Trainable params: 505 (1.97 KB)

 Non-trainable params: 40,000,100 (152.59 MB)

In [26]:
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [27]:
model.fit(X_train_sequences,Y_train_oh,epochs=50,batch_size=32,shuffle=True)

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1241 - loss: 1.6637  
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1029 - loss: 1.6498 
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1211 - loss: 1.6334 
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1191 - loss: 1.6257     
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1793 - loss: 1.5994  
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1445 - loss: 1.6185 
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1843 - loss: 1.5902 
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2173 - loss: 1.5933 
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2397 - loss: 1.5787 
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2175 - loss: 1.5801 
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2747 - loss: 1.5632 
Epoch 12/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2663 - loss: 1.5

In [28]:
x_test=np.array(['I am happy','not feeling happy','i want to die'])
x_test_indices=senetences_to_indices(x_test,words_to_index,max_len)
pred=model.predict(x_test_indices)

for item,label in zip(x_test,pred):
  print(item,label_to_emoji(np.argmax(label)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
I am happy 😄
not feeling happy 😞
i want to die 😞


In [29]:
from tensorflow.keras.layers import LSTM

def Emojify_V2(input_shape,word_to_vec_map,word_to_index):
  sentence_indices=Input(input_shape,dtype=np.int32)
  emb=pretrained_embedding_layer(word_to_vec_map,word_to_index)
  embeddings=emb(sentence_indices)

  X=LSTM(64,return_sequences=True)(embeddings)
  X=LSTM(64)(X)
  X=Dense(5,activation='softmax')(X)

  model=Model(sentence_indices,X)
  return model

In [30]:
model = Emojify_V2((max_len,),embeddings_index,words_to_index)

2024-07-28 19:24:19.241596: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 160000400 exceeds 10% of free system memory.


In [31]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 10, 100)        │    40,000,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 10, 64)         │        42,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,075,689 (152.88 MB)

 Trainable params: 75,589 (295.27 KB)

 Non-trainable params: 40,000,100 (152.59 MB)

In [32]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [33]:
model.fit(X_train_sequences,Y_train_oh,epochs=50,batch_size=32,shuffle=True)

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.2358 - loss: 1.5974
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2921 - loss: 1.5309
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.4308 - loss: 1.4709
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3773 - loss: 1.4190
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4761 - loss: 1.3322
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5302 - loss: 1.2357
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6778 - loss: 1.1338
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6917 - loss: 1.0002
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7749 - loss: 0.8629
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7350 - loss: 0.7434
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7624 - loss: 0.6771
Epoch 12/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8162 - loss: 0.5673
E

In [34]:
x_test=np.array(['I am happy','not feeling happy','i want to die'])
x_test_indices=senetences_to_indices(x_test,words_to_index,max_len)
pred=model.predict(x_test_indices)

for item,label in zip(x_test,pred):
  print(item,label_to_emoji(np.argmax(label)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 390ms/step
I am happy 😄
not feeling happy 😞
i want to die 😞


In [35]:
from tensorflow.keras.layers import LSTM,Bidirectional

def Emojify_V3(input_shape,word_to_vec_map,word_to_index):
  sentence_indices=Input(input_shape,dtype=np.int32)
  emb=pretrained_embedding_layer(word_to_vec_map,word_to_index)
  embeddings=emb(sentence_indices)

  X=Bidirectional(LSTM(64,return_sequences=True))(embeddings)
  X=Bidirectional(LSTM(64))(X)
  X=Dense(5,activation='softmax')(X)

  model=Model(sentence_indices,X)
  return model

In [36]:
model = Emojify_V3((max_len,),embeddings_index,words_to_index)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train_sequences,Y_train_oh,epochs=50,batch_size=32,shuffle=True)

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - accuracy: 0.2922 - loss: 1.5883
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.3899 - loss: 1.4805
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.3497 - loss: 1.3760
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5619 - loss: 1.2597
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.7341 - loss: 1.1392
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7277 - loss: 0.9749
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.7882 - loss: 0.8424
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8099 - loss: 0.6893
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8426 - loss: 0.5223
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8711 - loss: 0.4201
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8883 - loss: 0.2989
Epoch 12/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9222 - loss: 0.2430
E

In [37]:
x_test=np.array(['I am happy','not feeling happy','i want to die'])
x_test_indices=senetences_to_indices(x_test,words_to_index,max_len)
pred=model.predict(x_test_indices)

for item,label in zip(x_test,pred):
  print(item,label_to_emoji(np.argmax(label)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
I am happy 😄
not feeling happy 😞
i want to die 😞
